In [1]:
%matplotlib inline

import matplotlib as mpl
# mpl.use('TkAgg')
# import matplotlib.cm as cm
# from scipy.ndimage import filters as filters
from skimage import io, color, util
import numpy as np
# import math
import os
# import utils as custom_utils
import collections as coll
import sys

In [2]:
def get_k_centers(k, height, width):
    grid_y, grid_x = np.mgrid[:height, :width]
    slices = util.regular_grid((height, width), k)
    step_y, step_x = [int(s.step if s.step is not None else 1) for s in slices]
    segments_y = grid_y[slices]
    segments_x = grid_x[slices]
    
    centers = np.concatenate([segments_y[..., np.newaxis], segments_x[..., np.newaxis]], axis=-1)
    centers = centers.reshape(-1, 2)
    
    return centers, step_y, step_x

In [3]:
def get_window_bounds(center, step_y, step_x, height, width):
    cy = center[0]
    cx = center[1]

    y_min = int(max(cy - step_y, 0))
    y_max = int(min(cy + step_y + 1, height))
    x_min = int(max(cx - step_x, 0))
    x_max = int(min(cx + step_x + 1, width))
    return y_min, y_max, x_min, x_max

In [14]:
def generate_feature(point, image):
    color = np.zeros((image.shape[2],))
    feature = np.concatenate([color, point], axis=-1)
    for i in xrange(image.shape[2]):
        feature[i] = image[feature[-2], feature[-1]][i]
    return feature

In [15]:
def get_yx(feat):
    return feat[-2:]

In [16]:
def calculate_distance(vector, center):
#     print vector
#     print center
#     dy = (center[-2] - vector[-2]) ** 2
#     dist_center = (dy + (center[-1] - vector[-1]) ** 2) * spatial_weight

#     print dist_center
    
#     dist_color = 0
#     n_features = len(vector)
#     for c in range(0, n_features - 2):
#         dist_color += (vector[c] - center[c]) ** 2
        
#     print dist_color
#     dist_center += dist_color
#     print dist_center
#     return dist_center
    return np.sum((vector - center) ** 2)

In [7]:
def generate_features_vec(points, generate_feature, image):
    feature_vec = []
    for point in points:
        feature_vec.append(generate_feature(point, image))
    return np.asarray(feature_vec)    

In [8]:
# params
images = ['Images/Q3/castle.jpg']
n_segments = 100 # k
max_iter = 10

In [9]:
image = io.imread(images[0])
# image = color.rgb2lab(image)
height, width = image.shape[:2]
print image.shape

(313, 500, 3)


In [10]:
segments_yx, step_y, step_x = get_k_centers(n_segments, height, width)

In [11]:
n_centers = segments_yx.shape[0]

nearest_segments = np.empty((height, width), dtype=np.intp)
distance = np.empty((height, width), dtype=np.double)

n_segment_elems = np.zeros(n_centers, dtype=np.intp)

c_feat_all = generate_features_vec(segments_yx, generate_feature, image)
    
# for i in xrange(max_iter):
for i in xrange(1):
    change = 0
    distance[:, :] = sys.float_info.max
    for k in xrange(n_centers):
        c_feat = c_feat_all[k]
        c = get_yx(c_feat)
        y_min, y_max, x_min, x_max = get_window_bounds(c, step_y, step_x, height, width)
        for y in xrange(y_min, y_max):
            for x in xrange(x_min, x_max):
                yx_feat = generate_feature((y, x), image)
                dist = calculate_distance(yx_feat, c_feat)
                if dist < distance[y, x]:
                    nearest_segments[y, x] = k
                    distance[y, x] = dist
                    change = 1
                
    if change == 0:
        break

    n_segment_elems[:] = 0
    c_feat_all[:, :] = 0
    for y in xrange(height):
        for x in xrange(width):
            k = nearest_segments[y, x]
            n_segment_elems[k] += 1
            yx_feat = generate_feature((y, x), image)
            c_feat_all[k] += yx_feat
            
    for k in xrange(n_centers):
        c_feat_all[k] /= n_segment_elems[k]

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
# compactness=10.
# ratio = 1.0 / compactness